## 1.Dependencias

In [ ]:
!pip install --upgrade selenium

In [ ]:
!pip install http-request-randomizer

In [ ]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!which chromedriver

/usr/bin/chromedriver


In [ ]:
driver_path = "/usr/bin/chromedriver"

In [ ]:
import json
import time
import random
import string
import requests
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

## 2.Catálogos

In [ ]:
# Urls expuestas en la página, con estos se puede extraer las combinaciones de marca, año, modelo
# y variacion de los autos en comparabien

url_brand = "https://comparabien.com/services/pe/external/ws-brand.php"
url_anio = "https://comparabien.com/services/pe/external/ws-brand_year.php"
url_modelo = "https://comparabien.com/services/pe/external/ws-brand_model.php"
url_variacion = "https://comparabien.com/services/pe/external/ws-brand_model_var2.php"
url_precio = "https://comparabien.com/services/pe/external/ws-brand_model_precios2.php"

data_list = []

# Filtro de algunas marcas conocidas
brands_filtered = ["Audi", "BMW", "Chery", "Chevrolet", "Daihatsu", "Ford", "Honda", "Hyundai", "Jeep", "Kia", "Mazda", "Mercedes-Benz", "Mitsubishi", "Nissan", "Peugeot", "Renault", "SsangYong", "Subaru", "Suzuki", "Toyota", "Volkswagen", "Volvo"]
years_filtered = ["2016", "2017"]

lista_marca = [{"marca_id": d["id"], "marca": d["description"]} for d in requests.get(url_brand).json() if d["description"] in brands_filtered]
#lista_marca = [{'marca': 'Toyota', 'marca_id': '32'}]#, {'marca': 'Hyundai', 'marca_id': '14'}

# Proceso de extraccion de las combinaciones de autos
for marca_dict in lista_marca:
    print("marca: ", marca_dict["marca"])
    anio_list = requests.post(url_anio, data={"brand": marca_dict["marca_id"]}).json()
    anio_list = [d for d in anio_list if d["id"] in years_filtered]
    
    for anio_dict in anio_list:
        #print("\tanio: ", anio_dict["description"])
        modelo_list = requests.post(url_modelo, data={"brand": marca_dict["marca_id"],
                                                      "year": anio_dict["id"]
                                                     }).json()
        
        for modelo_dict in modelo_list[:2]:
            #print("\t\tmodelo: ", modelo_dict["description"])
            variacion_list = requests.post(url_variacion, data={"year": anio_dict["id"],
                                                                "model": modelo_dict["id"],
                                                               }).json()
            
            for variacion_dict in variacion_list:
                #print("\t\t\tvariacion: ", variacion_dict["description"])
                precio_dict = requests.post(url_precio, data={"year": anio_dict["id"],
                                                                "variante": variacion_dict["id"],
                                                               }).json()
                
                data_dict = {"marca": marca_dict["marca"],
                             "marca_id": marca_dict["marca_id"],
                             "anio": anio_dict["description"],
                             "modelo_id": modelo_dict["id"],
                             "modelo": modelo_dict["description"],
                             "variacion_id": variacion_dict["id"],
                             "variacion": variacion_dict["description"],
                             "precio": precio_dict["price"],
                             "precio_tipo": precio_dict["type"]
                }
                
                data_list.append(data_dict)

marca:  Audi
marca:  BMW
marca:  Chery
marca:  Chevrolet
marca:  Daihatsu
marca:  Ford
marca:  Honda
marca:  Hyundai
marca:  Jeep
marca:  Kia
marca:  Mazda
marca:  Mercedes-Benz
marca:  Mitsubishi
marca:  Nissan
marca:  Peugeot
marca:  Renault
marca:  SsangYong
marca:  Subaru
marca:  Suzuki
marca:  Toyota
marca:  Volkswagen
marca:  Volvo


In [ ]:
df = pd.DataFrame(data_list)
df

,marca,marca_id,anio,modelo_id,modelo,variacion_id,variacion,precio,precio_tipo
0,Audi,2,2017,0209,A1,020902,1.4 TFSI Si S tronic 125 hp,16000,1
1,Audi,2,2017,0209,A1,020901,1.4T FSI Sportback 125 hp,17500,1
2,Audi,2,2017,0201,A3,020102,1.4T Sportback,21000,1
3,Audi,2,2017,0201,A3,020108,1.8T FSI Sportback 180 ho,¡,1
4,Audi,2,2017,0201,A3,020118,2.0 Sedan TFSI quattro S tronic 290 hp,22000,1
...,...,...,...,...,...,...,...,...,...
253,Volvo,34,2016,3403,S60,340313,Polestar 2.0 370 hp,30800,1
254,Volvo,34,2016,3403,S60,340302,2.0T (2010) Confort,20000,1
255,Volvo,34,2016,3403,S60,340308,T4 Drive E Kinetic 2.0,22500,1
256,Volvo,34,2016,3403,S60,340304,3.0T6,20000,1


In [ ]:
data_dict = data_list[0]
data_dict

{'anio': '2017',
 'marca': 'Audi',
 'marca_id': '2',
 'modelo': 'A1 ',
 'modelo_id': '0209',
 'precio': '16000',
 'precio_tipo': '1',
 'variacion': '1.4 TFSI Si S tronic 125 hp',
 'variacion_id': '020902'}

In [ ]:
# Valores de planes, combustible y ubicacion. Por defecto se tomaran los primeros valores es decir. Todo Riesgo, Gasonina/Diesel y Lima y Callao.
plan_list = ['Todo Riesgo', 'Básica', 'Pérdida Total']
combustible_list = ['Gasolina/Diesel', 'GLP/GNV', 'Bicombustible']
ubicacion_list = ['Lima y Callao', 'Amazonas', 'Ancash', 'Apurímac', 'Arequipa', 'Ayacucho', 'Cajamarca', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín', 'La Libertad', 'Lambayeque', 'Loreto', 'Madre de Dios', 'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes', 'Ucayali']

## 3.Extrayendo data

#### Proxies

In [ ]:
from bs4 import BeautifulSoup

response = requests.get("http://free-proxy-list.net")
content = response.content
soup = BeautifulSoup(content, "html.parser")
table = soup.find("table", attrs={"class": "table table-striped table-bordered"})
headings = [th.get_text() for th in table.find("tr").find_all("th")]

ip_list = []
for row in table.find_all("tr")[1:-1]:
    dataset = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
    ip_list.append(dataset)

In [ ]:
df_proxy = pd.DataFrame(ip_list)
america_proxy_countries = ["Brazil", "Peru", "Colombia", "Bolivia", "Venezuela", "Ecuador", "Mexico"]
df_proxy = df_proxy.loc[df_proxy["Country"].isin(america_proxy_countries)].reset_index(drop=True)
ip_list = df_proxy.to_dict(orient="records")

In [ ]:
ip_list = ["200.60.94.155:5678", "200.107.163.242:5678", "200.10.73.210:5678", "181.66.37.98:4153"]

#### User Agents

In [ ]:
user_agent_list = [d["user_agent"] for d in pd.read_csv("user_agents.txt", header=None, names=["user_agent"]).to_dict(orient='records')]

#### Funciones

In [ ]:
def random_email():
    return ''.join(random.choice(string.ascii_letters) for _ in range(random.randint(1, 20))) + "@gmail.com"

def random_phone():
    return str(random.randint(991234567, 999999999))

def get_select(driver, option_value, waiting_time=4):
    wait = WebDriverWait(driver, waiting_time)
    option_element = wait.until(ec.presence_of_element_located((By.CSS_SELECTOR, "option[value='{value}']".format(value=option_value))))
    select_element = Select(option_element.find_element(By.XPATH, ".."))
    
    return select_element

def get_data_from_table_row(driver, xTmp):
    xTmp2 = xTmp.find_element(By.CLASS_NAME, "cell.parent")
    xTmp3 = xTmp2.find_elements(By.CLASS_NAME, "cell.mob-mid.alt")[-1]
    xTmp4 = xTmp2.find_element(By.CLASS_NAME, "coberturas")
    xTmp5 = xTmp.find_element(By.XPATH, ".//form[@action='/seguros-vehiculares/capture']")

    #producto = xTmp.find_element(By.XPATH, ".//div[@class='name hide-mob']").text
    porcentaje_deducible = xTmp3.find_elements(By.TAG_NAME, "span")[0].text
    pago_minimo_deducible = driver.execute_script('return arguments[0].childNodes[3].textContent;', xTmp3).strip()
    coberturas_dict = {e.find_element(By.CLASS_NAME, "seg-cob-tit").text: e.find_element(By.CLASS_NAME, "seg-cob-dato").text for e in driver.execute_script('return arguments[0].childNodes;', xTmp4) if len(e.find_elements(By.CLASS_NAME, "seg-cob-tit")) > 0}
    prima_anual = xTmp.find_element(By.XPATH, ".//span[@class='key-figure']").text
    prima_mensual = xTmp.find_element(By.XPATH, ".//span[@class='small']").text
    request_input = {e.get_attribute("name"): e.get_attribute("value") for e in xTmp5.find_elements(By.XPATH, ".//input[@type='hidden']")}

    data_dict = {#"producto": producto, <- ya se encuentra en input_request como prod_name
                 "porcentaje_deducible": porcentaje_deducible,
                 "pago_minimo_deducible": pago_minimo_deducible,
                 "coberturas_dict": coberturas_dict,
                 "prima_anual": prima_anual,
                 "prima_mensual": prima_mensual,
                 "request_input": request_input
                }
    
    return data_dict

def get_products(driver):
    try:
      result_table = WebDriverWait(driver, 5).until(ec.presence_of_element_located((By.ID,'result')))
      
      quantity = WebDriverWait(driver, 5).until(ec.text_to_be_present_in_element((By.XPATH, "//div[@class='top-data container']"), "opciones disponibles"))
      print("\tOptions available: ", quantity)
      
      data_list = [get_data_from_table_row(driver, e) for e in result_table.find_elements(By.CLASS_NAME, "item")]
    except TimeoutException:
      data_list = []

    return data_list

def extract_data(driver, data_dict):
    try:
        close_button = WebDriverWait(driver, 5).until(ec.presence_of_element_located((By.XPATH, "//i[@class='fas fa-times-circle fa-lg vehiculo-close']")))
        if close_button:
            driver.execute_script("arguments[0].click();", close_button)
    except (NoSuchElementException, TimeoutException) as e:
        pass
    
    form_seguro = driver.find_element(By.ID, "compare")
    
    ## Seleccionando el modelo de auto
    marca_button = WebDriverWait(driver, 5).until(ec.presence_of_element_located((By.XPATH,".//button[@data-id='brand']")))
    marca_button.click()

    brand_select = get_select(driver, data_dict["marca_id"])
    brand_select.select_by_visible_text(data_dict["marca"])
    
    year_select = get_select(driver, data_dict["anio"])
    year_select.select_by_value(data_dict["anio"])
    
    model_select = get_select(driver, data_dict["modelo_id"])
    model_select.select_by_value(data_dict["modelo_id"])
    
    variation_select = get_select(driver, data_dict["variacion_id"])
    variation_select.select_by_value(data_dict["variacion_id"])
    
    ## Seleccionando el plan de cobertura. Por defecto es 'Todo Riesgo'
    plan_select = Select(form_seguro.find_element(By.XPATH, ".//select[@id='cover']"))
    plan_select.select_by_visible_text(plan_list[0])

    ## Seleccionando el tipo de combustible. Por defecto es 'Gasolina/Diesel'
    combustible_select = Select(form_seguro.find_element(By.XPATH, ".//select[@id='combustible']"))
    combustible_select.select_by_visible_text(combustible_list[0])

    ## Seleccionando la ubicacion. Por defecto es 'Lima y Callao'
    location_select = Select(form_seguro.find_element(By.XPATH, ".//select[@id='geo']"))
    location_select.select_by_visible_text(ubicacion_list[0])
    
    ## Ingresando el numero de telefono aleatorio
    phone_field = driver.find_element(By.ID, "phone")
    phone_field.clear()
    phone_field.send_keys(random_phone())
    
    ## Ingresando el email aleatorio
    email_field = driver.find_element(By.ID, "email-field")
    email_field.clear()
    email_field.send_keys(random_email())
    
    ## Click en Comparar para obtener resultados
    compare_button = WebDriverWait(driver, 5).until(ec.presence_of_element_located((By.ID,'boton-compara')))
    compare_button.click()
    
    ## Click en boton de no especificar recorrido de auto.
    try:
        km_button = WebDriverWait(driver, 5).until(ec.presence_of_element_located((By.XPATH, "//button[@class='btn btn-secondary btn-modal-no btn-modal-dni-enviar']")))
        km_button.click()
    except (NoSuchElementException, TimeoutException) as e:
        pass
    
    ## Llamando a la funcion de extraccion de datos de productos.
    data = get_products(driver)
    
    ## Cerrando la busqueda del modelo, para iniciar una nueva.
    close_button = WebDriverWait(driver, 5).until(ec.presence_of_element_located((By.XPATH, "//i[@class='fas fa-times-circle fa-lg vehiculo-close']")))
    driver.execute_script("arguments[0].click();", close_button)
    
    return data

def get_insurance_data(data_dict):
    
    #proxy_dict = random.choice(ip_list)
    proxy_value = random.choice(ip_list)
    
    options = Options()
    options.add_argument("--headless") #
    options.add_argument('--no-sandbox')
    options.add_argument("--disable-extensions")
    options.add_argument('start-maximized')
    options.add_argument('disable-infobars')
    options.add_argument('user-agent='+random.choice(user_agent_list))
    #options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36')
    #options.add_argument('--proxy-server=%s' % proxy_value)#proxy_dict["IP Address"] + ":" + proxy_dict["Port"]
    
    service = Service(driver_path)
    
    driver = webdriver.Chrome(service=service, options=options)#
    driver.get(url)
    print("\tUser agent: {0}".format(driver.execute_script("return navigator.userAgent;")))
    
    extracted_data_dict = {"extracted_data": extract_data(driver, data_dict)}
    
    extracted_data_dict.update(data_dict)
    extracted_data_dict["plan_input"] = "Todo Riesgo"
    extracted_data_dict["combustible_input"] = "Gasolina/Diesel"
    extracted_data_dict["ubicacion_input"] = "Lima y Callao"

    driver.quit()
    
    return extracted_data_dict

In [ ]:
url = "https://comparabien.com.pe/seguros-vehiculares"

## 4. Test

In [ ]:
extracted_data_list = []
for data_dict in data_list[:10]:
    print(data_dict)
    extracted_data_dict = get_insurance_data(data_dict)
    extracted_data_list.append(extracted_data_dict)

{'marca': 'Audi', 'marca_id': '2', 'anio': '2017', 'modelo_id': '0209', 'modelo': 'A1 ', 'variacion_id': '020902', 'variacion': '1.4 TFSI Si S tronic 125 hp', 'precio': '16000', 'precio_tipo': '1'}
	User agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0
	Options available:  True
{'marca': 'Audi', 'marca_id': '2', 'anio': '2017', 'modelo_id': '0209', 'modelo': 'A1 ', 'variacion_id': '020901', 'variacion': '1.4T FSI Sportback 125 hp', 'precio': '17500', 'precio_tipo': '1'}
	User agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36
	Options available:  True
{'marca': 'Audi', 'marca_id': '2', 'anio': '2017', 'modelo_id': '0201', 'modelo': 'A3', 'variacion_id': '020102', 'variacion': '1.4T  Sportback', 'precio': '21000', 'precio_tipo': '1'}
	User agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36
	Options availa

In [ ]:
pd.DataFrame(extracted_data_list)

,extracted_data,marca,marca_id,anio,modelo_id,modelo,variacion_id,variacion,precio,precio_tipo,plan_input,combustible_input,ubicacion_input
0,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0209,A1,020902,1.4 TFSI Si S tronic 125 hp,16000,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
1,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0209,A1,020901,1.4T FSI Sportback 125 hp,17500,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
2,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0201,A3,020102,1.4T Sportback,21000,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
3,[],Audi,2,2017,0201,A3,020108,1.8T FSI Sportback 180 ho,¡,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
4,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0201,A3,020118,2.0 Sedan TFSI quattro S tronic 290 hp,22000,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
5,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0201,A3,020107,1.6 Sportback,18500,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
6,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0201,A3,020109,2.0 FSI Sportback 190 hp,24000,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
7,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0201,A3,020110,2.0T FSI Sportback,28000,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
8,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0201,A3,020106,1.2 Sedan 110 hp Attracion,24500,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao
9,"[{'porcentaje_deducible': '15', 'pago_minimo_d...",Audi,2,2017,0201,A3,020104,1.8 T FSI S tronic 180 hp,17500,1,Todo Riesgo,Gasolina/Diesel,Lima y Callao


In [ ]:
print(json.dumps(extracted_data_list[:2], indent=4, sort_keys=True))

[
    {
        "anio": "2017",
        "combustible_input": "Gasolina/Diesel",
        "extracted_data": [
            {
                "coberturas_dict": {
                    "": "",
                    "Gastos M\u00e9dicos:": "$ 4,000 por persona",
                    "Muerte / Invalidez:": "$ 25,000 por persona",
                    "Resp. Civil a 3ros:": "$ 150,000"
                },
                "pago_minimo_deducible": "min: $ 500",
                "porcentaje_deducible": "15",
                "prima_anual": "$ 727",
                "prima_mensual": "(12 cuotas de $ 60.61)",
                "request_input": {
                    "agente": "",
                    "agente_email": "",
                    "agno": "2017",
                    "auto_brand": "2",
                    "auto_model": "0209",
                    "comp_id": "45",
                    "comp_img": "logos/positiva.png",
                    "comp_name": "La Positiva Seguros",
                    "costo_auto"